In [2]:
import bempp.api
import numpy as np
from decimal import *

import bemppUQ
from bemppUQ.config import config
from bemppUQ.operators.maxwell import assemble_operators, evaluate_far_field

from bemppUQ.utils.mie import bhmie
bempp.api.global_parameters.assembly.potential_operator_assembly_type = 'dense'

ImportError: cannot import name 'coefficients_from_grid_functions_list'

In [ ]:
k_ext, k_int = config["k_ext"], config["k_int"]
lambda_par, freq = config["lambda"], config["frequency"]
polarization = config["polarization"]
direction = config["direction"]

eps_rel = config["eps_rel"]
mu_rel = config["mu_rel"]
print("The exterior wavenumber is: {0}".format(k_ext))
print("The interior wavenumber is: {0}".format(k_int))

print("----")
print("The exterior wavelenght is: {0}".format(lambda_par))
print("The exterior frequency is: {:.2E}".format(Decimal(freq)))


In [ ]:
precision = 10

h = 2.0 * np.pi / (precision * config["k_int"])

grid = bempp.api.shapes.sphere(h=h)
k_int, k_ext = config['k_int'], config['k_ext']
print(grid.leaf_view.entity_count(1))
n = k_int / k_ext
refIndex = n
numAngles = 901
s1,s2,qext,qsca,qback,gsca=bhmie(k_ext,k_int / k_ext,numAngles)
transmission_operators = assemble_operators(grid, config)
far_field, solution = evaluate_far_field(transmission_operators, config)

In [ ]:
from matplotlib import pyplot as plt

angles = config['angles']
k_ext = config['k_ext']

fig, ax  = plt.subplots(1,1, figsize=(6,6))
A22 = far_field[2,:]   # -1j * kExt is to make agree with eqn 3.21 p70 tBohren and Huffman

plt.rcParams['figure.figsize'] = (15, 4) # Increase the figure size in the notebook

uh = abs(4 * np.pi  / (-1j * k_ext) * s1)**2
u = abs(A22[:1801])**2

ax.semilogy(angles[:1801], uh, '-r', label='BEM solution')
ax.semilogy(angles[:1801], u, '--b', label='Analytic solution')

ax.set_title(r'$\mathrm{RCS}_z(\theta)$: BEM (red) vs. analytic (blue)', fontsize=14)

ax.set_xlabel(r'$\theta$ (rad)', fontsize=14)
ax.set_ylabel(r'$\mathrm{RCS}_z(\theta)$', fontsize=14)


ymin, ymax = uh.min() * .7 , uh.max() * 1.3
ax.legend()
ax.grid()
plt.xlim(-.1, np.pi + 0.1)
plt.ylim(ymin, ymax)
plt.savefig('plots/RCSSphere.pdf')
plt.show() 

In [ ]:
precision_list = [1, 2, 5, 10, 20, 30, 40, 50,60]

for precision in precision_list:
    h = 2*np.pi/(precision*k_ext)
    print('Starting run for precision: ', precision)
    
    grid = bempp.api.shapes.sphere(h=h)
    N =  2 * grid.leaf_view.entity_count(1)
    print('Ndof: ', N)
    bempp.api.global_parameters.hmat.eps = 1E-4
    transmission_operators = assemble_operators(grid, config)
    far_field, solution = evaluate_far_field(transmission_operators, config)
    
    A22 = - 1j * k_ext * far_field[2,:]
    name = (
    "results/"
        + "precision"
    + str(precision)
    )

    my_dict = {
            "A22": A22,
            "h": h,
            "N": N,        
        }

    np.save(name + ".npy", my_dict)        